In [1]:
from py2neo import Graph
model = Graph

In [2]:
graph=Graph(uri="bolt://localhost:7687",auth=("neo4j","1234"))

In [3]:
q = """
MATCH (p) return (p) LIMIT 10
"""

data = graph.run(q)

l=[]
for d in data:
    l.append(d)

In [4]:

q1="""
MATCH (p)-[:MAKES_PRODUCT]-(m:Product)-[:IMPORTED_FROM]-(c:Country)
WHERE m.sector='healthcare' AND m.type="Masks" AND c.name="China"
RETURN p
"""

data1 = graph.run(q1)

l1=[]
for d1 in data1:
    l1.append(d1)

In [5]:

q2="MATCH (p)-[:MAKES_PRODUCT]-(m)-[:IMPORTED_FROM]-(c:Country) WHERE m.sector='healthcare' RETURN p"

data2 = graph.run(q2)
l2=[]
for d2 in data2:
    l2.append(d2)

In [6]:
q3= """
MATCH (n)-[:MAKES_PRODUCT]-(m)-[:IMPORTED_FROM]-(c:Country)
WHERE m.sector='healthcare' AND m.type="Masks"
RETURN c.name, count(*) as high_dependence_country
ORDER BY high_dependence_country DESC;
"""
data3 = graph.run(q3)

l3=[]
for d3 in data3:
    l3.append(d3) 

In [7]:
#Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

from tensorflow.python.framework import ops
ops.reset_default_graph()

#UContextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from keras import backend as K

import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [8]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bhima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
words = []
classes = []
documents = []
ignore_words = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))


In [12]:

training = []
output = []
output_empty = [0] * len(classes)


for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [13]:

random.shuffle(training)
training = np.array(training)


train_x = list(training[:,0])
train_y = list(training[:,1])


In [14]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training....


In [15]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [16]:

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

model.save('model.tflearn')


Training Step: 2999  | total loss: 0.00430 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.00430 - acc: 1.0000 -- iter: 16/19
Training Step: 3000  | total loss: 0.00420 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.00420 - acc: 1.0000 -- iter: 19/19
--
INFO:tensorflow:C:\Users\Bhima\NLP-Neo4j IR\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [17]:

pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [18]:

data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    

# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Bhima\NLP-Neo4j IR\model.tflearn


In [19]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    rand_response = random.choice(i['responses'])
                    return str(rand_response)

            results.pop(0)

In [20]:
import pickle
filename='model.pkl'
pickle.dump(tflearn.DNN, open(filename, 'wb'))

In [ ]:
while True:
    input_data = input("You-")
    answer = response(input_data)
    #answer
    if answer == "limit":
        print(*l, sep ="\n")
    if answer == "masks":
        print(*l1, sep ="\n")
    if answer == "health":
        print(*l2, sep ="\n")
    if answer == "dependentmasks":
        print(*l3, sep ="\n")

### 